In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
# prompt: read the csv /content/MBTI 500.csv and /content/mbti_1.csv

import pandas as pd
df1 = pd.read_csv('/content/drive/MyDrive/MBIT_data/MBTI 500.csv')
df2 = pd.read_csv('/content/drive/MyDrive/MBIT_data/mbti_1.csv')


In [ ]:
df1.describe()

In [ ]:
df2.describe()

In [ ]:
df2.head()

In [ ]:
df1.head()

In [ ]:
df2.head()

In [ ]:
# prompt: count null in df1

df1.isnull().sum()


In [ ]:
df1 = df1.dropna()

In [ ]:
df2.isnull().sum()

In [ ]:
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from concurrent.futures import ThreadPoolExecutor
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the Porter Stemmer
port_stem = PorterStemmer()
stop_words = set(stopwords.words('english'))

# Optimized stemming function
def stemming(content):
    # Remove non-alphabetical characters and convert to lower case
    content = re.sub('[^a-zA-Z]', ' ', content).lower()
    # Tokenize and remove stopwords
    words = [port_stem.stem(word) for word in content.split() if word not in stop_words]
    # Join words back into a single string
    return ' '.join(words)

# Function to process a chunk
def process_chunk(chunk):
    chunk['posts'] = chunk['posts'].apply(stemming)
    return chunk

# Split the DataFrame into 20 chunks
num_chunks = 200
chunks = np.array_split(df1, num_chunks)

# Use ThreadPoolExecutor to process chunks in parallel
with ThreadPoolExecutor(max_workers=num_chunks) as executor:
    results = list(executor.map(process_chunk, chunks))

# Combine the results back into a single DataFrame
df1_stemmed = pd.concat(results)

# Vectorize the text data using TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df1_stemmed['posts'])

# X is now the term-document matrix


In [ ]:
import re
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from concurrent.futures import ThreadPoolExecutor
from sklearn.feature_extraction.text import TfidfVectorizer
from queue import Queue
import threading

# Initialize the Porter Stemmer
port_stem = PorterStemmer()
stop_words = set(stopwords.words('english'))

# Optimized stemming function
def stemming(content):
    # Remove non-alphabetical characters and convert to lower case
    content = re.sub('[^a-zA-Z]', ' ', content).lower()
    # Tokenize and remove stopwords
    words = [port_stem.stem(word) for word in content.split() if word not in stop_words]
    # Join words back into a single string
    return ' '.join(words)

# Function to process a chunk and report progress
def process_chunk(chunk, progress_queue, chunk_id):
    chunk['posts'] = chunk['posts'].apply(stemming)
    progress_queue.put(chunk_id)
    return chunk

# Split the DataFrame into 20 chunks
num_chunks = 20
chunks = np.array_split(df1, num_chunks)

# Queue to track progress
progress_queue = Queue()
lock = threading.Lock()

def monitor_progress(total_chunks):
    processed_chunks = 0
    while processed_chunks < total_chunks:
        chunk_id = progress_queue.get()
        with lock:
            processed_chunks += 1
            print(f"Thread finished processing chunk {chunk_id}. Progress: {processed_chunks}/{total_chunks}")

# Start the progress monitoring in a separate thread
monitor_thread = threading.Thread(target=monitor_progress, args=(num_chunks,))
monitor_thread.start()

# Use ThreadPoolExecutor to process chunks in parallel
with ThreadPoolExecutor(max_workers=num_chunks) as executor:
    futures = {executor.submit(process_chunk, chunk, progress_queue, i): i for i, chunk in enumerate(chunks)}

# Wait for all futures to complete
for future in futures:
    future.result()

# Combine the results back into a single DataFrame
df1_stemmed = pd.concat(future.result() for future in futures)

# Ensure the progress monitoring thread finishes
monitor_thread.join()

# Vectorize the text data using TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df1_stemmed['posts'])

# Save the vectorized matrix X to Google Drive
drive_path = '/content/drive/MyDrive/MBIT_data/'

with open(drive_path + 'vectorized_matrix.pkl', 'wb') as file:
    pickle.dump(X, file)

# Save the vectorizer to Google Drive
with open(drive_path + 'vectorizer.pkl', 'wb') as file:
    pickle.dump(vectorizer, file)

print("Vectorized matrix and vectorizer saved successfully to Google Drive.")


In [ ]:
import pickle

# Define the path to your Google Drive
drive_path = '/content/drive/MyDrive/MBIT_data/'

# Save the vectorized matrix X
with open(drive_path + 'vectorized_matrix.pkl', 'wb') as file:
    pickle.dump(X, file)

# Save the vectorizer
with open(drive_path + 'vectorizer.pkl', 'wb') as file:
    pickle.dump(vectorizer, file)

print("Vectorized matrix and vectorizer saved successfully to Google Drive.")


In [ ]:

# Define the path to your Google Drive
drive_path = '/content/drive/MyDrive/MBIT_data/'

# Load the vectorized matrix X
with open(drive_path + 'vectorized_matrix.pkl', 'rb') as file:
    X = pickle.load(file)

# Load the vectorizer
with open(drive_path + 'vectorizer.pkl', 'rb') as file:
    vectorizer = pickle.load(file)

print("Vectorized matrix and vectorizer loaded successfully from Google Drive.")


In [ ]:
def stemming(content):
    stemmed_content=re.sub('[^a-zA-Z]',' ',content)
    stemmed_content=stemmed_content.lower()
    stemmed_content=stemmed_content.split()
    stemmed_content=[port_stem.stem(words) for words in stemmed_content if not words in stopwords.words('english')]
    stemmed_content=' '.join(stemmed_content)
    return stemmed_content

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
from concurrent.futures import ThreadPoolExecutor
def process_chunk(chunk):
    chunk['posts'] = chunk['posts'].apply(stemming)
    return chunk

# Split the DataFrame into 20 chunks
num_chunks = 20
chunks = np.array_split(df1, num_chunks)

# Use ThreadPoolExecutor to process chunks in parallel
with ThreadPoolExecutor(max_workers=num_chunks) as executor:
    results = list(executor.map(process_chunk, chunks))

# Combine the results back into a single DataFrame
df1_stemmed = pd.concat(results)

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


KeyboardInterrupt: 

In [ ]:
df1['posts']=df1['posts'].apply(stemming)

In [ ]:
x=df1['posts'].values
y=df1['type'].values